# Plan
Ok, so my plan is basically just to break the dataset specifically the date time into different catagorical values. Specifically two new coluns, first the year, a numerical value in order to account for the possibility of a raising or lowering of avg temperachure over a few years, and a second a value 1-356 that gives which day of the specified year it is. I think that by doing it this way the model will be able to diferentiate between the different times of the year without having to deal with wierd time series and all that crazy stuff. Basically I just need to find a model that will train off of that data based on the individual values.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wids-all-datasets/train_data.csv
/kaggle/input/wids-all-datasets/test_data.csv
/kaggle/input/wids-all-datasets/sample_solution.csv


In [2]:
df=pd.read_csv('/kaggle/input/wids-all-datasets/train_data.csv')
df=df.dropna(axis=1)

In [3]:
#Make the dataframe shorter
df=df[0:100]

In [4]:
#Function to convert dataframe to prefered format.
def df_format_neumerical(df):
    df['dayofyear']=0
    df['year']=0
    i=0
    while i<len(df):
        day=df['startdate'][i]
        period=pd.Period(day, freq='D')
        df['dayofyear'][i]=int(period.day_of_year)
        df['year'][i]=int(period.year)
        i+=1
    df.pop('startdate')
    neumerical_df=df[:]
    neumerical_df.pop('climateregions__climateregion')
    return df,neumerical_df

In [5]:
df,neumerical_df=df_format_neumerical(df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Ok, so I think that with that I am ready to actually start on the model that I am trying to train.

In [6]:
#df=pd.read_csv('/kaggle/input/wids-all-datasets/train_data.csv')
#df.dropna()

Tuturial I am going to use:https://machinelearninggeek.com/multi-layer-perceptron-neural-network-using-python/

In [7]:
# Import LabelEncoder
from sklearn import preprocessing

# Creating labelEncoder
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
df['climateregions__climateregion']=le.fit_transform(df['climateregions__climateregion'])

In [8]:
#Sepperate dataframe into test and neumerical data
def seperate_train(df):
    seventy_cutoff=int(.7*len(df))
    ninty_cutoff=int(len(df)-(.1*len(df)))
    train_neumerical_df=neumerical_df[:seventy_cutoff]
    test_neumerical_df=neumerical_df[ninty_cutoff:len(df)]
    return train_neumerical_df, test_neumerical_df

In [9]:
train_neumerical_df,test_neumerical_df=seperate_train(df)

In [10]:
def df_seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [11]:
y_train_neumerical_df,X_train_neumerical_df=df_seperate_y(train_neumerical_df)
y_test_neumerical_df, X_test_neumerical_df=df_seperate_y(test_neumerical_df)

In [12]:
y_train_neumerical_df=y_train_neumerical_df.astype('int')

In [13]:
# Import MLPClassifer 
from sklearn.neural_network import MLPClassifier

# Create model object
clf = MLPClassifier(hidden_layer_sizes=(2400,2000),
                    random_state=100,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train_neumerical_df,y_train_neumerical_df)

Iteration 1, loss = 36.04554292
Iteration 2, loss = 31.41144482
Iteration 3, loss = 29.86684693
Iteration 4, loss = 32.95647399
Iteration 5, loss = 32.44176963
Iteration 6, loss = 33.47181934
Iteration 7, loss = 30.11383640
Iteration 8, loss = 30.68588066
Iteration 9, loss = 30.59076686
Iteration 10, loss = 26.86793416
Iteration 11, loss = 25.06656038
Iteration 12, loss = 14.83838039
Iteration 13, loss = 24.04033936
Iteration 14, loss = 24.20545381
Iteration 15, loss = 31.93552661
Iteration 16, loss = 25.23572924
Iteration 17, loss = 29.87013253
Iteration 18, loss = 26.26598428
Iteration 19, loss = 29.35566005
Iteration 20, loss = 24.72170211
Iteration 21, loss = 28.50115653
Iteration 22, loss = 21.88945549
Iteration 23, loss = 23.92582466
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(2400, 2000), learning_rate_init=0.01,
              random_state=100, verbose=True)

In [14]:
y_test_neumerical_df=y_test_neumerical_df.astype('int')

In [15]:
ypred=clf.predict(X_test_neumerical_df)

# Import accuracy score 
from sklearn.metrics import accuracy_score

# Calcuate accuracy
accuracy_score(y_test_neumerical_df,ypred)
#I'll have to make a specialized function to do this

0.0

In [16]:
ypred

array([15, 15, 15, 15, 15, 15, 15, 15, 15, 15])

In [17]:
y_test_neumerical_df

90    18
91    18
92    18
93    18
94    18
95    18
96    18
97    17
98    17
99    18
Name: contest-tmp2m-14d__tmp2m, dtype: int64

In [18]:
#Trying to create a final 
final=pd.DataFrame()
final['contest-tmp2m-14d__tmp2m']=ypred
final['# index']=0
i=len(neumerical_df)-len(X_test_neumerical_df)
while i<len(df):
    final['# index'][i-(len(neumerical_df)-len(X_test_neumerical_df))]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,# index
0,15,90
1,15,91
2,15,92
3,15,93
4,15,94
5,15,95
6,15,96
7,15,97
8,15,98
9,15,99


In [19]:
final.to_csv('testfinal.csv',index=False)

In [20]:
final

,contest-tmp2m-14d__tmp2m,# index
0,15,90
1,15,91
2,15,92
3,15,93
4,15,94
5,15,95
6,15,96
7,15,97
8,15,98
9,15,99
